In [ ]:
# Make sure that you have set up API KEYS in Secrets on the left menu icon.
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
!pip install -q python-dotenv langchain openai langchain-openai faiss-gpu langchain-community

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain.chains import create_retrieval_chain

In [ ]:
def get_documents_from_web(url):
    loader = WebBaseLoader(url)
    docs = loader.load()

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,
        chunk_overlap=20
    )
    splitDocs = splitter.split_documents(docs)
    return splitDocs

In [ ]:
def create_db(docs):
    embedding = OpenAIEmbeddings()
    vectorStore = FAISS.from_documents(docs, embedding=embedding)
    return vectorStore

In [ ]:
def create_chain(vectorStore):
    model = ChatOpenAI(
        model="gpt-3.5-turbo-1106",
        temperature=0.4,
        openai_api_key=os.getenv('OPENAI_API_KEY')
    )
    prompt = ChatPromptTemplate.from_template("""
    Answer the user's question:
    Context: {context}
    Question: {input}
    """)

    # chain = prompt | model
    chain = create_stuff_documents_chain(
        llm=model,
        prompt=prompt
    )
    retriever = vectorStore.as_retriever(search_kwargs={"k": 1})
    retrieval_chain = create_retrieval_chain(
        retriever,
        chain
    )
    return retrieval_chain

In [ ]:
docs = get_documents_from_web('https://python.langchain.com/docs/expression_language/')
vectorStore = create_db(docs)
chain = create_chain(vectorStore)

In [ ]:
response = chain.invoke({
    "input": "What is LCEL?",
})

print(response["context"])